In [1]:
#load data
import pandas as pd
from scipy import sparse
from collections import defaultdict
from scipy import sparse
from sklearn import model_selection as cv
import numpy as np

header = ["reviewerId","productId","rating","time"]
df = pd.read_csv('ratings_Video_Games.csv', names=header)
#df = pd.read_csv('item_dedup.csv', names=header)
n_users = df.reviewerId.unique().shape[0]
n_items = df.productId.unique().shape[0]
print ('Number of users = ' + str(n_users) + ' | Number of items = ' + str(n_items))


train_data, test_data = cv.train_test_split(df, test_size=0)

# Create dict for product id 
#df.productId.unique()
productDict = defaultdict(dict)
userDict = defaultdict(dict)
count = 0


for id in train_data.reviewerId.unique():
    userDict[id] = count
    count += 1

count = 0
for id in train_data.productId.unique():
    productDict[id] = count
    count += 1
    
n_users = len(userDict)
n_items = len(productDict)

print(n_users, " ",  n_items,"\n")

mat = sparse.lil_matrix((n_users, n_items),dtype=np.float32)
        
print(productDict['B00004XOLO'])
print(userDict['A2EHIE51TERF2C'])


for line in train_data.itertuples():
    mat[userDict[line[1]],productDict[line[2]]] = line[3]

print(mat.getnnz)
print("done")

Number of users = 826767 | Number of items = 50210
826767   50210 

12105
269005
<bound method lil_matrix.getnnz of <826767x50210 sparse matrix of type '<class 'numpy.float32'>'
	with 1324753 stored elements in LInked List format>>
done


In [3]:
#Joseph and Jack
import pandas as pd
import gzip

def parse(path):
    print("parsing")
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    print("getting df")
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF('meta_Video_Games.json.gz')
print("done: meta_df holds all item metadata")

getting df
parsing
done: meta_df holds all item metadata


In [5]:
#Joseph and Jack

#this code cell is just used to play around with accessing the data
print( meta_df.iloc[0], "\n_______________\n") #printing the first element of the df
singleItem = meta_df.loc[meta_df['asin'] == 'B000066TS5']
print(singleItem,"\n_______________\n" ) #printing the info of the item with the specified item ID
for item in singleItem.iterrows():
    print(item[1][4], "\n_______________\n") #['also_bought']
    print(item[1][4]['also_bought'], "\n_______________\n")
    print(item[1][4]['also_bought'][0], "\n_______________\n")
    print(item[1][6], "\n_______________\n")

asin                                                  0078764343
description                                    Brand new sealed!
price                                                      37.98
imUrl          http://ecx.images-amazon.com/images/I/513h6dPb...
related        {'also_bought': ['B000TI836G', 'B003Q53VZC', '...
salesRank                                 {'Video Games': 28655}
categories                      [[Video Games, Xbox 360, Games]]
title                                                        NaN
brand                                                        NaN
Name: 0, dtype: object 
_______________

            asin                                        description  price  \
7599  B000066TS5  Ever wondered what would happen if the worlds ...   52.5   

                                                  imUrl  \
7599  http://ecx.images-amazon.com/images/I/51MtbYNR...   

                                                related             salesRank  \
7599  {'also_boug

In [7]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import sklearn.preprocessing as pp

def cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=0)
    print(col_normed_mat)
    return col_normed_mat.T * col_normed_mat

def user_cosine_similarities(mat):
    col_normed_mat = pp.normalize(mat, axis=0)
    #print(col_normed_mat)
    return col_normed_mat * col_normed_mat.T


#cosine_similarities(mat)
#test = cosine_similarities(mat)
#userMatrix = user_cosine_similarities(mat)
#user_similarity = pairwise_distances(mat, metric='cosine')
print("done")
#print(test.getnnz)
#print(userMatrix.getnnz)


#cx = sparse.coo_matrix(test)

#for i,j,v in zip(cx.row, cx.col, cx.data):
#        if v > 0.5:
#            print ("(%d, %d), %s" % (i,j,v))



done


In [3]:
# https://stackoverflow.com/questions/31790819/scipy-sparse-csr-matrix-how-to-get-top-ten-values-and-indices
#print(test[50208,40340])

row = test.getrow(6565).toarray()[0].ravel()
top_ten_indicies = row.argsort()[-6:]
top_ten_values = row[row.argsort()[-6:]]

print(top_ten_indicies)

print(top_ten_values)

#for x in top_ten_indicies:
#    print(list(productDict.keys())[list(productDict.values()).index(x)])
    


#for x in range(0,n_items):
#    print(x)
#    row = test.getrow(x).toarray()[0].ravel()
#    top_ten_values = (row[row.argsort()[-10:]])

print("done")



#array = test.getrow(i)
#print(array)


[ 7571  8531 13135 18165 29636  6565]
[0.03652888 0.03891427 0.03941018 0.07974918 0.12854701 0.9999998 ]
done


In [4]:
def getTopFiveSimiliarItems(itemId):
    row = test.getrow(itemId).toarray()[0].ravel()
    top_five_indicies = row.argsort()[-6:]
    top_five_values = row[row.argsort()[-6:]]
    print(top_five_values)
    print(top_five_indicies)
    for x in top_five_indicies:
        #ignore same item
        if (x != itemId):
            print(list(productDict.keys())[list(productDict.values()).index(x)])
        
getTopFiveSimiliarItems(9331)
getTopFiveSimiliarItems(productDict["B000EBC0PI"])
    

[0.10657972 0.10899496 0.12630129 0.15185843 0.19566512 0.99999994]
[11338 36740 32618 34937 14826  9331]
B00005V5N1
B00005V6BZ
B00000K4Y7
B000066BZ5
B00005V3MB
[0.         0.03449096 0.04490133 0.06150693 0.30411437 1.        ]
[16725   367 20371  1852 21161 50171]
B001CXYDQQ
B004FYEZMQ
B0047O2Q0M
B000K8ZTK8
B0007PIE9C


In [5]:
print(df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"])
print("A")
line = df.loc[df['reviewerId'] == "A2UDD2XDPD80BH"]

for x in line.iterrows(): #finding all items rated by the user
    #print(x[1][1])
    print(productDict[x[1][1]])
    getTopFiveSimiliarItems(productDict[x[1][1]])
    #print("======================")
    
#df.loc[df['reviewerId'] == "A10RFAF1CGFKHZ"]
print("AA")
print(list(productDict.keys())[list(productDict.values()).index(1076)], "\nB") 
print(list(productDict.keys())[list(productDict.values()).index(23340)], "\nC") 
print(list(userDict.keys())[list(userDict.values()).index(76565)], "\n") 

            reviewerId   productId  rating        time
124881  A2UDD2XDPD80BH  B000066TS5     1.0  1160956800
198242  A2UDD2XDPD80BH  B0001I1LD6     5.0  1145664000
204696  A2UDD2XDPD80BH  B00020LZAW     5.0  1142553600
228327  A2UDD2XDPD80BH  B00067AVT2     5.0  1142208000
237315  A2UDD2XDPD80BH  B0007PLKZM     5.0  1160092800
258699  A2UDD2XDPD80BH  B0009WPZM2     5.0  1159142400
270903  A2UDD2XDPD80BH  B000B43OXU     5.0  1144540800
284015  A2UDD2XDPD80BH  B000BI3QK2     5.0  1161648000
285552  A2UDD2XDPD80BH  B000BLM5PG     5.0  1156809600
289093  A2UDD2XDPD80BH  B000C427NM     5.0  1155427200
289669  A2UDD2XDPD80BH  B000CBVC0Y     5.0  1170979200
304679  A2UDD2XDPD80BH  B000FBMITA     5.0  1149206400
319037  A2UDD2XDPD80BH  B000FRS9II     5.0  1167436800
A
3305
[0.05525838 0.05794913 0.06040753 0.06185607 0.14857282 1.0000049 ]
[ 985 2325  852 1197 3450 3305]
B00008URUA
B001G3B0HC
B00005TNI6
B00068O278
B0009A4EVM
39258
[0.08253928 0.0927837  0.09755657 0.10920884 0.18762553 1.    

In [11]:
#Joseph and Jack
def getItemInfoVector(itemID): #counts the number of 1, 2, 3, 4, and 5 star ratings and returns an array of those totals (for a single ID)
    ratings = [0, 0, 0, 0, 0] #1,2,3,4,5 (so that if you want the number of n star ratings, you look at index n-1)
    reviewList = df.loc[df['productId'] == itemID]
    #print( reviewList, "\n_______\n")
    for item in reviewList.iterrows():
        itemRating = item[1][2]
        if( itemRating == 5):
            ratings[4]+=1
        elif( itemRating == 4):
            ratings[3]+=1
        elif( itemRating == 3):
            ratings[2]+=1
        elif( itemRating == 2):
            ratings[1]+=1
        elif( itemRating == 1):
            ratings[0]+=1
    #print( itemID, "'s ratings (1-5 stars): ",ratings)
    return ratings
    #print(list(productDict.keys())[list(productDict.values()).index(itemId)])
    

In [154]:
#Joseph and Jack
#getItemInfoVector("B000066TS5")
from scipy import spatial


def vectorCosineSimilarity( baseVector, otherVector):
    badVector = [0,0,0,0,0]
    if otherVector == badVector:
        return 0
    return 1-spatial.distance.cosine(baseVector, otherVector)

def findItemBFS( relatedItemID, depth, baseVector, explored): #returns a tuple with (cos_sim, itemID)
    bestItem = (0, relatedItemID) 
    if relatedItemID in explored: #dupeCheck
        return bestItem
    otherVector = getItemInfoVector(relatedItemID)
    bestItem =( vectorCosineSimilarity(baseVector, otherVector), relatedItemID)
    #print( bestItem)
    #depth check here
    if depth == 0:
        return bestItem, explored
    #afterwords, go through related items
    itemMetaData = meta_df.loc[meta_df['asin'] == relatedItemID]
    for metaItem in itemMetaData.iterrows(): #fake for loop with only one item
        for relatedItem in metaItem[1][4]['also_bought']: #real for loop to go through all related items
            #print( relatedItem )
            if relatedItem in explored:
                continue
            bfsResult, explored = findItemBFS( relatedItem , depth-1, baseVector, explored)
            if bfsResult[0] > bestItem[0]:
                bestItem = bfsResult
            if relatedItem not in explored:
                explored.append( relatedItem )
    return bestItem, explored

def getMostSimilarItem( item ): #finds the item with the most similar cosine similarity
    exploredList = [item[1][1]] #to keep track of visited items
    bestItem = (0, item[1][1]) #a tuple: (cos_sim, itemID)
    #print( item[1][1])
    mainVector = getItemInfoVector(item[1][1]) # the vector we are doing the cosine similarity with
    mainItemMetaData = meta_df.loc[meta_df['asin'] == item[1][1]]
    for metaItem in mainItemMetaData.iterrows(): #fake for loop with only one item
        print(metaItem[1][0])
        print(metaItem[1][4])
        for relatedItem in metaItem[1][4]['also_bought']: #real for loop (TODO: handle cases where also_bought is not there)
            #print( relatedItem )
            if relatedItem in exploredList:
                continue
            bfsResult, exploredList = findItemBFS( relatedItem , 0, mainVector, exploredList)
            if bfsResult[0] > bestItem[0]:
                bestItem = bfsResult
            if relatedItem not in exploredList:
                exploredList.append( relatedItem )
    #print( item[1][1], " ", mainVector)
    return bestItem
    

In [155]:
#Joseph and Jack
def recommendItems( userID ): #the main function (takes in a user ID)
    userReviewList = df.loc[df['reviewerId'] == userID]
    print( "User's Item List: \n", userReviewList)
    resultsList = []
    counter = 0
    print( "\nSearching for similar items...")
    for item in userReviewList.iterrows():
        if counter > 1:
            break #for debugging purposes
        #print( item[1][1]) #item[1][1] is the productId
        #print( item[1][1], " ", getItemInfoVector(item[1][1]))
        resultsList.append( getMostSimilarItem( item))
        print( "Found similar item ", counter)
    return resultsList

In [156]:
recommendItems("A2UDD2XDPD80BH" )

User's Item List: 
             reviewerId   productId  rating        time
124881  A2UDD2XDPD80BH  B000066TS5     1.0  1160956800
198242  A2UDD2XDPD80BH  B0001I1LD6     5.0  1145664000
204696  A2UDD2XDPD80BH  B00020LZAW     5.0  1142553600
228327  A2UDD2XDPD80BH  B00067AVT2     5.0  1142208000
237315  A2UDD2XDPD80BH  B0007PLKZM     5.0  1160092800
258699  A2UDD2XDPD80BH  B0009WPZM2     5.0  1159142400
270903  A2UDD2XDPD80BH  B000B43OXU     5.0  1144540800
284015  A2UDD2XDPD80BH  B000BI3QK2     5.0  1161648000
285552  A2UDD2XDPD80BH  B000BLM5PG     5.0  1156809600
289093  A2UDD2XDPD80BH  B000C427NM     5.0  1155427200
289669  A2UDD2XDPD80BH  B000CBVC0Y     5.0  1170979200
304679  A2UDD2XDPD80BH  B000FBMITA     5.0  1149206400
319037  A2UDD2XDPD80BH  B000FRS9II     5.0  1167436800

Searching for similar items...
B000066TS5
{'also_bought': ['B0009A4EVM', 'B001G3B0HC', 'B000TLU67W', 'B001MW91IW', 'B00004YRQA', 'B0000C7GHG', '0744001986', '9629971372', 'B00005R5PO', 'B0013KW1B2', 'B00004YRQ

KeyError: 'also_bought'